<h1> Overview <h1>

<h2> Project Set-up <h2>

In [1]:
pkg_variables <- function(status){
    pkg_list = c("ape","biomformat","corrplot","cowplot","cluster",
             "data.table","expss",
             "GGally","Hmisc","ggpubr","ggnetwork","ggplot2","grid",
             "gridExtra",
             "otuSummary",
             "plyr","RColorBrewer",
             "scales","tibble","vegan","yaml","dplyr","tidyverse","phyloseq")

    for (pkg in pkg_list){
        print(pkg)
        #Detaching
        if(status=="detach"){
            print("detaching")
            
            #If the package has been loaded detach
            if(pkg %in% (.packages())){
                pkg_name = paste("package", pkg, sep = ":")
                detach(pkg_name, unload = TRUE, character.only = TRUE)   
            }
        }
        else{
            lapply(pkg, require, character.only = TRUE)
        }
    }
}

In [136]:
library(extrafont)
extrafont::loadfonts(device="win")
windowsFonts(Times=windowsFont("TT Times New Roman"))

#pkg_variables(status="detach")
pkg_variables(status="attach")

source("sources/ggrare.R") #github library: https://rdrr.io/github/gauravsk/ranacapa/
source("sources/miseqR.R")
source("sources/qiime2R.R") #github https://github.com/jbisanz/qiime2R
source('sources/abundances.R') #github https://github.com/microbiome/microbiome
source('sources/ordination.R') #github https://github.com/umerijaz/microbiomeSeq
source('sources/plot.ordination.R') #github https://github.com/umerijaz/microbiomeSeq
source('sources/plot_anova_diversity.R') #github https://github.com/umerijaz/microbiomeSeq
source('sources/alpha_div.R')
source('sources/perform_anova.R')

Agency FB already registered with windowsFonts().

Algerian already registered with windowsFonts().

Arial Black already registered with windowsFonts().

Arial already registered with windowsFonts().

Arial Narrow already registered with windowsFonts().

Arial Rounded MT Bold already registered with windowsFonts().

Bahnschrift already registered with windowsFonts().

Baskerville Old Face already registered with windowsFonts().

Bauhaus 93 already registered with windowsFonts().

Bell MT already registered with windowsFonts().

Berlin Sans FB already registered with windowsFonts().

Berlin Sans FB Demi already registered with windowsFonts().

Bernard MT Condensed already registered with windowsFonts().

Blackadder ITC already registered with windowsFonts().

Bodoni MT already registered with windowsFonts().

Bodoni MT Black already registered with windowsFonts().

Bodoni MT Condensed already registered with windowsFonts().

Bodoni MT Poster Compressed already registered with windowsFon

Pristina already registered with windowsFonts().

Rage Italic already registered with windowsFonts().

Ravie already registered with windowsFonts().

Rockwell already registered with windowsFonts().

Rockwell Condensed already registered with windowsFonts().

Rockwell Extra Bold already registered with windowsFonts().

Script MT Bold already registered with windowsFonts().

Segoe MDL2 Assets already registered with windowsFonts().

Segoe Print already registered with windowsFonts().

Segoe Script already registered with windowsFonts().

Segoe UI already registered with windowsFonts().

Segoe UI Light already registered with windowsFonts().

Segoe UI Semibold already registered with windowsFonts().

Segoe UI Semilight already registered with windowsFonts().

Segoe UI Black already registered with windowsFonts().

Segoe UI Emoji already registered with windowsFonts().

Segoe UI Historic already registered with windowsFonts().

Segoe UI Symbol already registered with windowsFonts().

Show

[1] "ape"
[1] "biomformat"
[1] "corrplot"
[1] "cowplot"
[1] "cluster"
[1] "data.table"
[1] "expss"
[1] "GGally"
[1] "Hmisc"
[1] "ggpubr"
[1] "ggnetwork"
[1] "ggplot2"
[1] "grid"
[1] "gridExtra"
[1] "otuSummary"
[1] "plyr"
[1] "RColorBrewer"
[1] "scales"
[1] "tibble"
[1] "vegan"
[1] "yaml"
[1] "dplyr"
[1] "tidyverse"
[1] "phyloseq"


In [3]:
run_param="dceg"
#run_param="diss"

In [4]:
proj_dir='/Users/slsevilla/Google Drive/MyDocuments_Current/Education/George Mason University/Dissertation/Data/Aim1/'
out_dir=paste(proj_dir,'output/',sep="")
img_dir=paste(proj_dir,out_dir,'img/',sep="")

<h3> Control and metadata input <h3>

In [5]:
#Read in taxonomy file
CONTdf = read.csv('../taxonomy/Taxonomy_Complete.csv',stringsAsFactors = FALSE)

#update df to use the updated controls
CONTdf = subset(CONTdf,select = -c(D6310,D6300))
if(run_param=="dceg"){
    CONTdf = CONTdf %>% 
      rename(
        M22 = DZ35322,
        M16 = DZ35316,
        A00 = MSA1000,
        A01 = MSA1001,
        A02 = MSA1002,
        A03 = MSA1003,
        Z00 = D6300_Updated,
        Z05 = D6305,
        Z06 = D6306,
        Z10 = D6310_Updated,
        Z11 = D6311
    )
} else{
    CONTdf = CONTdf %>% 
      rename(
        D6300 = D6300_Updated,
        D6310 = D6310_Updated
    )
}

#Convert the tax file to SILVA naming with "D_5__"
CONTdf_s = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("D_5__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("D_4__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Family"] = new
}

In [6]:
if(run_param=="dceg"){
        metadata<-read.table('../manifest/m_cleaned_dceg.txt',sep='\t', header=T, row.names=1, comment="")
} else{
        metadata<-read.table('../manifest/m_cleaned.txt',sep='\t', header=T, row.names=1, comment="")
}

<h2> Create PhyloSeq Object<h2>

In [7]:
phylo_create <- function(tax_in,tab_in){
    #Read OTUS
    otus<-read_qza(tab_in)

    #Read Greengenes taxonomy file
    taxonomy<-read_qza(tax_in)

    #Create taxonomy table
    tax_table = data.frame()
    col = 1
    row = 1

    for (i in taxonomy$data$Taxon){
        tax_list = as.list(strsplit(as.character(i),";"))
        for (i in 1:lengths(tax_list)){
            tax_table[row,col] = tax_list[[1]][i]
            col = col + 1
        }
        col = 1
        row = row + 1
    }
    rownames(tax_table)<-taxonomy$data$Feature.ID
    colnames(tax_table)<-c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
    tax_convert<-as.matrix(tax_table)  
    
    #Create phylo object
    phy_obj<-phyloseq(otu_table(otus$data, taxa_are_rows = T), tax_table(tax_convert), sample_data(metadata))
    print(phy_obj) 
    return(phy_obj)
}   

In [8]:
phy_subset_ext <- function(phy_in){
    #glom to tax level, subset for only seq controls
    phy_obj = tax_glom(phy_in,taxrank="Genus") #only merges genera that have the same upper taxonomy
    phy_obj = subset_samples(phy_obj,Sample.Type=="Ext_Control" & Sample.Des!="Ext_Blank")
    return(phy_obj)
}

phy_subset_blank <- function(phy_in){
    #glom to tax level, subset for only seq controls
    phy_obj = tax_glom(phy_in,taxrank="Genus") #only merges genera that have the same upper taxonomy
    phy_obj = subset_samples(phy_obj,Sample.Des=="Ext_Blank")
    return(phy_obj)
}

In [9]:
otu_extract <- function(phy_in){
    # Extract abundance matrix from the phyloseq object
    OTU1 = as(otu_table(phy_in), "matrix")
    if(taxa_are_rows(phy_in)){OTU1 <- t(OTU1)} # transpose
    OTUdf = as.data.frame(OTU1)# Coerce to data.frame
    return(OTUdf)
}

In [10]:
tax_extract <- function(phy_in){
    #extract taxonomy information into dataframe
    TAXdf = tax_table(phy_in)
    return(TAXdf)
}

In [11]:
otu_tax_clean <- function(OTU_in,TAX_in){
    #Remove zero cols
    OTUdf_clean = OTU_in[, which(colSums(OTU_in) != 0)]

    #rename col from features to genus
    for (i in colnames(OTU_in)){
        newname = paste(as.character(TAX_in[i,"Family"]),as.character(TAX_in[i,"Genus"]),sep="")
        
        #standardize gg naming
        newname = trimws(newname) #remove first space in gg naming
        newname = sub(" g__","g__",newname)
        newname = sub("\\[","",newname)
        newname = sub("\\]","",newname)
        
        if(is.na(newname)){
            next;
        } else{
            names(OTUdf_clean)[names(OTUdf_clean) == i] <- newname
        }
    }
    
    #add metadata col
    for (a in rownames(OTUdf_clean)){
        OTUdf_clean[a,"Subject.ID"] = metadata[a,"Subject.ID"]
        OTUdf_clean[a,"Ext.Kit"] = metadata[a,"Ext.Kit"]
        OTUdf_clean[a,"Homo.Status"] = metadata[a,"Homo.Status"]
    }
    OTUdf_clean = data.frame(OTUdf_clean,stringsAsFactors=FALSE)
    
    return(OTUdf_clean)
}

<h2> TP,FP,FN <h2>

In [12]:
#Create TP,FP,FN dataframe
bench_compare <- function(OTUdf_in,CONTdf_in){
    
    F1df = OTUdf_in
    F1df$Sample_names = rownames(F1df) #create col of sample_names
    
    #For each of the controls in the cleaned df
    for (cont in unique(OTUdf_in$Subject.ID)){
        temp=filter(CONTdf_in, get(cont) !=0) #create df of taxa related to control
        
        #Create taxa list, merging family and genus - this will help avoid id errors from taxonomic differences
        #IE one taxa labeled at the genus level may be from family1 and the same taxa may also be from family2
        taxa_list=list()
        for (x in rownames(temp)){taxa_list = c(taxa_list,paste(temp[x,"Family"],temp[x,"Genus"],sep=""))}  
        
        #Create list of samples that match the control
        sample_list = filter(F1df, Subject.ID == cont)$Sample_names
        
        #If the data is not in the df (FN), then add the taxa as a new col
        for (taxa in taxa_list){ 
            if(!(taxa %in% unique(colnames(F1df)))){
                F1df[taxa[!(taxa %in% colnames(F1df))]] = 0
            } 
        }

        for (sample in sample_list){
            for (col in unique(colnames(F1df))[unique(colnames(F1df)) != "Subject.ID" & unique(colnames(F1df)) != "Sample_names"
                & unique(colnames(F1df)) != "Ext.Kit" & unique(colnames(F1df)) != "Homo.Status"]){ 
                #removes subjectID, sample_names cols from loop
                value_check = F1df[sample,col]

                if(col %in% taxa_list && value_check > 0){ #column matches expected taxa, has features
                    F1df[sample,col] = "TP"
                } else if (col %in% taxa_list && value_check == 0){ #column matches expected taxa, no features
                    F1df[sample,col] = "FN"
                } else if ((!col %in% taxa_list) && value_check > 0){ #column doesnt match expected taxa, has features
                    F1df[sample,col] = "FP"
                } else{ #column doesnt match expected taxa, has no features
                    next
                }
            }
        }
    }
    return(F1df)
}

In [13]:
#Create counts of FP, FN, TP
bench_counts <- function(OTUdf_in,F1_in,bmark_in,var_in){
    F1df_counts = subset(OTUdf_in, select=c("Subject.ID"))

    for (sample in rownames(F1df)){
        FN = rowSums(F1_in=="FN")[sample][[1]]
        FP = rowSums(F1_in=="FP")[sample][[1]]
        TP = rowSums(F1_in=="TP")[sample][[1]]

        F1df_counts[sample,"check"]= FN + TP
        F1df_counts[sample,"FN"] = FN
        F1df_counts[sample,"FP"] = FP
        F1df_counts[sample,"TP"] = TP
        F1df_counts[sample,"TAR"] = TP / (TP + FP)
        F1df_counts[sample,"TDR"] = TP / (TP + FN)

        num = F1df_counts[sample,"TAR"]*F1df_counts[sample,"TDR"]
        den = F1df_counts[sample,"TAR"]+F1df_counts[sample,"TDR"] 
        F1df_counts[sample,"F1"] = 2*(num/den)
        F1df_counts[sample,"Subject.ID"] = OTUdf_in[sample,"Subject.ID"]
        
        F1df_counts[sample,"bmark"] = bmark_in
        F1df_counts[sample,"bmark2"] = var_in
    }
    return(F1df_counts)
}

<h3> Plotting <h3>

In [14]:
plot_violin <- function(df_in,x_lab,y_ax,y_lab,col_in,y_lowerlim,y_upperlim){
    if(y_lowerlim=="" | y_upperlim==""){
        p = ggplot(df_in, aes(x=get(x_lab), y=get(y_ax), color=get(col_in))) + 
            geom_violin(trim=FALSE) + 
            labs(color = col_in) +
            ylab(y_lab) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") +
            stat_summary(fun.data=mean_sdl, color="red")
    } else{
        p = ggplot(df_in, aes(x=get(x_lab), y=get(y_ax), color=get(col_in))) + 
            geom_violin(trim=FALSE) + 
            labs(color = col_in) +
             ylab(y_lab) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") +
            ylim(y_lowerlim,y_upperlim) +
            stat_summary(fun.data=mean_sdl, color="red")
    }    
    return (p)
}

In [15]:
plot_f1_violin <- function(F1_in,x_list,y_list,id_in,title,ftype,y_lowerlim="",y_upperlim=""){
    
    p_1 = plot_violin(F1_in,id_in,x_list[1],y_list[1],id_in,y_lowerlim,y_upperlim)
    p_2 = plot_violin(F1_in,id_in,x_list[2],y_list[2],id_in,y_lowerlim,y_upperlim)
    p_3 = plot_violin(F1_in,id_in,x_list[3],y_list[3],id_in,y_lowerlim,y_upperlim)

    #Create count violin plots
    fig = ggarrange(p_1, p_2, p_3, 
            labels = c("A", "B", "C"),
            ncol = 2, nrow = 2)
    
    annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                   top = text_grob(title, color = "blue", face = "bold", size = 14)
    )
    
    ggsave(paste(out_dir,"/img/violin_f1_",ftype,"_",title,".jpg",sep=""))
}

In [16]:
plot_violin_hmo <- function(df_in,x_lab,y_ax,y_lab,col_in,fac_in){
    if(fac_in==2){
        p = ggplot(df_in, aes(x=Subject.ID, y=get(y_ax), color=get(col_in))) + 
            facet_grid(Homo.Status~ get(x_lab))+
            geom_violin(trim=FALSE) + 
            labs(color = col_in) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            ylab(y_lab) +
            rotate_x_facet_text(angle = 90, align = 0.5) +
            stat_summary(fun.data=mean_sdl, color="red")
    } else if(fac_in==3){
        p = ggplot(df_in, aes(x=Homo.Status, y=get(y_ax), color=get(col_in))) + 
            facet_grid(Subject.ID ~get(x_lab))+
            geom_violin(trim=FALSE) + 
            labs(color = col_in) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            ylab(y_lab) +
            rotate_x_facet_text(angle = 90, align = 0.5) +
            stat_summary(fun.data=mean_sdl, color="red")    
        
    } else{
        p = ggplot(df_in, aes(x=get(x_lab), y=get(y_ax), color=get(col_in))) + 
            facet_grid(Subject.ID~ .)+
            geom_violin(trim=FALSE) + 
            labs(color = col_in) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            ylab(y_lab) +
            rotate_x_facet_text(angle = 90, align = 0.5) +
            stat_summary(fun.data=mean_sdl, color="red")
    }
    return (p)
}

In [17]:
rotate_x_facet_text <- function(angle = 45, align = 0, valign = 0.25) {
    ggplot2::theme(strip.text.x = ggplot2::element_text(angle = angle,
                                                        hjust = align,
                                                        vjust = valign))
    }

In [18]:
plot_f1_violin_hmo <- function(F1_in,x_list,y_list,id_in,title,ftype,fac_in=2){
    
    p_1 = plot_violin_hmo(F1_in,id_in,x_list[1],y_list[1],id_in,fac_in)
    annotate_figure(p_1, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                    top = text_grob(title, color = "blue", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/violin_f1_",ftype,"_",title,"_",x_list[1],".jpg",sep=""))
    
    p_2 = plot_violin_hmo(F1_in,id_in,x_list[2],y_list[2],id_in,fac_in)
    annotate_figure(p_2, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                    top = text_grob(title, color = "blue", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/violin_f1_",ftype,"_",title,"_",x_list[2],".jpg",sep=""))
    
    
    p_3 = plot_violin_hmo(F1_in,id_in,x_list[3],y_list[3],id_in,fac_in)
    annotate_figure(p_3, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                    top = text_grob(title, color = "blue", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/violin_f1_",ftype,"_",title,"_",x_list[3],".jpg",sep=""))
}

<h3> Relative Abunandance <h3>

In [19]:
cal_relabun <- function(df_in){
    temp = df_in
    temp$Total = rowSums(df_in[,1:(ncol(df_in)-3)]) #increased from -1 to -3
    col_list = colnames(temp[,1:(ncol(temp)-4)])
    
    for (row in rownames(temp)){
        for (col in col_list){
            val = temp[row,col]
            if(val==0){
                next
            } else{
                temp[row,col] = val/temp[row,"Total"]
            }
        }
    }
    return(temp)
}

<h2> Expected to Observed Correlation <h2>

In [20]:
cal_EO_kit <- function(df_in,contdf_in,bench_in,vari_in,title_in){
    count=1
    plot_list = list()
    EOdf_merged = data.frame()
    eo_out = data.frame()

    #For each of the controls in the cleaned df
    for (cont in unique(df_in$Subject.ID)){
        
        #Create df of taxa
        taxdf = create_taxadf(cont,contdf_in)
        taxa_list=taxdf$taxa.id #list of expected taxa for this control
        
        #Create list of samples that match the control
        sample_list = filter(RAdf, Subject.ID == cont)$Sample_name
        EOdf= data.frame()
        
        #for each expected taxa create df of sample name, taxa id, control, exp RA and obs RA
        for (taxa in taxa_list){
            for(sample in sample_list){
                EOdf[nrow(EOdf)+1,"Sample_name"] = sample
                EOdf[nrow(EOdf),"taxa.id"] = taxa
                EOdf[nrow(EOdf),"Subject.ID"] = cont
                EOdf[nrow(EOdf),"exp"] = taxdf[taxa,cont]
                EOdf[nrow(EOdf),"bench"] = bench_in

                if(is.null(df_in[sample,taxa])){ #if that sample doesn't have that taxa, obs is 0
                     EOdf[nrow(EOdf),"obs"] = 0
                } else{
                    EOdf[nrow(EOdf),"obs"] = df_in[sample,taxa] #otherwise observed val saves
                }
            }
        }
        EOdf_merged = rbind(EOdf_merged,EOdf) #Merge dfs
        
        #Create scatter plots
        plot_list[[count]] = plot_EO(EOdf,cont)
        count=count+1
        
        #Calc stats
        EOdf[is.na(EOdf)] = 0
        print(bench_in)
        temp = cal_EO_stattest(EOdf,cont,bench_in) #calc stats - answers Q is expected sig different from observed per cont?
        eo_out = rbind(eo_out,temp) #merge df       
    }
    
    #Plot controls
    plot_length = as.numeric(length(plot_list))
    if(plot_length==4){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]],
                            labels = c("A", "B", "C", "D"), #edit number of controls
                            ncol = 2, nrow = 3)  
    } else if(plot_length==3){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]],
                        labels = c("A", "B", "C"), #edit number of controls
                        ncol = 2, nrow = 3)
    } else if(plot_length==2){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], 
                        labels = c("A", "B"), #edit number of controls
                        ncol = 2, nrow = 3)
    } else{
        fig = ggarrange(plot_list[[1]],
                        labels = c("A"), #edit number of controls
                        ncol = 2, nrow = 3)

    }
    annotate_figure(fig, top = text_grob(title_in, color = "blue", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/scatter_ext",vari_in,"_",bench_in,".jpg",sep=""))
    
    return(eo_out)
}

In [21]:
cal_EO_stattest <- function(df_in,cont_in,bench_in){
    my_data=subset(df_in,Subject.ID==cont_in)
    df_out=data.frame()
    
    df_out[nrow(df_out)+1,"bmark"] = bench_in
    df_out[nrow(df_out),"cont"] = cont_in
    df_out[nrow(df_out),"shapiro.p"] = shapiro.test(my_data$obs)$p.value
    df_out[nrow(df_out),"shapiro.sig"] = if(df_out[nrow(df_out),"shapiro.p"]<0.05){"sig"}else{"ns"}
    df_out[nrow(df_out),"spear.p"] = cor.test(my_data$obs,my_data$exp,method = "spearman")$p.value
    df_out[nrow(df_out),"spear.rho"] = cor.test(my_data$obs,my_data$exp,method = "spearman")$estimate[[1]]
    df_out[nrow(df_out),"spear.sig"] = if(df_out[nrow(df_out),"spear.p"]<0.05){"sig"}else{"ns"}
    df_out[nrow(df_out),"pears.val"] = cor.test(my_data$obs,my_data$exp,method = "pearson")$estimate[[1]]
    df_out[nrow(df_out),"pears.p"] = cor.test(my_data$obs,my_data$exp,method = "pearson")$p.value
    df_out[nrow(df_out),"pears.sig"] = if(df_out[nrow(df_out),"pears.p"]<0.05){"sig"}else{"ns"}
    return(df_out)
}

In [22]:
create_taxadf <- function(cont,contdf_in){
    taxdf=filter(contdf_in, get(cont) !=0) #create df of taxa related to control
    taxdf$taxa.id=paste(taxdf$Family,taxdf$Genus,sep="") #create fam/genus col
    taxdf = taxdf[,c("taxa.id",cont),drop=FALSE] #remove all cols but the merged and control col
    taxdf = aggregate(get(cont) ~ taxa.id, data = taxdf, sum) #collapse df for any dups    
    names(taxdf)[2] = cont
    rownames(taxdf) = taxdf$taxa.id
    return(taxdf)
}

In [23]:
cal_EO_cont <- function(df_in,contdf_in,taxdf_in,cont_in,title_in){
    count=1
    plot_list = list()
    EOdf_merged = data.frame()
    eo_out = data.frame()
    
    #Create list of samples that match the control
    sample_list = filter(df_in, Subject.ID == cont_in)$Sample_name
    EOdf= data.frame()
    taxa_list=taxdf_in$taxa.id
    
    #for (kit in unique(df_in$Ext.Kit)){
        for (taxa in taxa_list){
            for(sample in sample_list){
                EOdf[nrow(EOdf)+1,"Sample_name"] = sample
                EOdf[nrow(EOdf),"taxa.id"] = taxa
                EOdf[nrow(EOdf),"Subject.ID"] = cont
                EOdf[nrow(EOdf),"exp"] = taxdf_in[taxa,cont]
                EOdf[nrow(EOdf),"bench"] = df_in[sample,"Ext.Kit"]

                if(is.null(df_in[sample,taxa])){
                    next
                } else{
                    EOdf[nrow(EOdf),"obs"] = df_in[sample,taxa]
                }
            }
        }
    #}

    for(kit in unique(EOdf$bench)){
        EOdf_sub = subset(EOdf,bench==kit)
        #Create scatter plots
        if(nrow(EOdf_sub)>1){
            plot_list[[count]] = plot_EO(EOdf_sub,cont_in,kit)
        count=count+1
        }
    }
    EOdf_merged = rbind(EOdf_merged,EOdf) #Merge dfs

    #Calc stats
    #EOdf[is.na(EOdf)] = 0
    #temp = cal_EO_stattest(EOdf,cont,bench_in)
    #eo_out = rbind(eo_out,temp) #merge df 
    plot_EO_save(plot_list,title_in,"std",cont_in)
    
    return(eo_out)
}

In [24]:
plot_EO <- function(df_in,cont_in,title=""){
    if(title==""){
        title=cont_in
        my_data=subset(df_in,Subject.ID==cont_in)
    } else{
        my_data=df_in
    }
    
    p = ggscatter(my_data, x = "obs", y = "exp", 
              add = "reg.line", conf.int = TRUE, 
              cor.coef = TRUE, cor.method = "pearson",
              xlab = "Expected Rel \n Abunance (%)", ylab = "Observed Rel \n Abun (%)") +
        ggtitle(paste("Scatter plot of Rel Abun:\n",title,sep=""))
    return(p)
}

In [25]:
plot_EO_save <- function(plot_list,title_in,vari_in,bench_in){
    #Plot controls
    plot_length = as.numeric(length(plot_list))
    if(plot_length==7){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]],plot_list[[5]],plot_list[[6]],plot_list[[7]],
                            labels = c("A", "B", "C", "D","E","F","G"), #edit number of controls
                            ncol = 2, nrow = 4)  
    }else if(plot_length==6){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]],plot_list[[5]],plot_list[[6]],
                            labels = c("A", "B", "C", "D","E","F"), #edit number of controls
                            ncol = 2, nrow = 3)  
    }else if(plot_length==5){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]],plot_list[[5]],
                            labels = c("A", "B", "C", "D","E"), #edit number of controls
                            ncol = 2, nrow = 3)  
    }else if(plot_length==4){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]],
                            labels = c("A", "B", "C", "D"), #edit number of controls
                            ncol = 2, nrow = 3)  
    } else if(plot_length==3){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]],
                        labels = c("A", "B", "C"), #edit number of controls
                        ncol = 2, nrow = 3)
    } else if(plot_length==2){
        fig = ggarrange(plot_list[[1]], plot_list[[2]], 
                        labels = c("A", "B"), #edit number of controls
                        ncol = 2, nrow = 3)
    } else{
        fig = ggarrange(plot_list[[1]],
                        labels = c("A"), #edit number of controls
                        ncol = 2, nrow = 3)

    }
    annotate_figure(fig, top = text_grob(title_in, color = "blue", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/scatter_ext",vari_in,"_",bench_in,".jpg",sep=""))
}

<h2>Dissimilarity <h2>

In [26]:
#Create averaged relative abundance df
create_av_RAdf <- function(RAdf_in,bench_in,var_in,status="std"){
    if(status=="std"){
        tmp = aggregate(RAdf_in[,1:(ncol(RAdf_in)-5)], list(RAdf_in$Subject.ID), mean) #increase from 3 to 5
        tmp$bmark=bench_in
        tmp$bmark2 = var_in
    } else{
        tmp = aggregate(RAdf_in[,1:(ncol(RAdf_in)-5)], list(RAdf_in$Subject.ID,RAdf_in$Homo.Status), mean)
        tmp$bmark=bench_in
        tmp$bmark2 = var_in
    }
    return(tmp)
}

<h3> Merged metrics <h3>

In [27]:
#Add expected values to RA average dfs
RAdf_addcontrols<-function(cont_in,df_in,hmo_in="std"){
    
    tmp=get(df_in)
    cont_list = unique(tmp$Group.1)
    cont_leng = length(cont_list) #chnaged to Group.1
    
    for(i in 1:cont_leng){
        cont = as.character(cont_list[i])
        taxadf = create_taxadf(cont,get(cont_in))
        tmp_count = nrow(tmp)+1
        
        if(hmo_in=="hmo"){
            for(tx_row in 1:nrow(taxadf)){
                tmp[tmp_count,"Group.1"]=cont
                tmp[tmp_count,taxadf[tx_row,"taxa.id"]] = taxadf[tx_row,cont]
                tmp[tmp_count,"bmark"]="expected"
                tmp[tmp_count,"bmark2"]=tmp[i,"bmark2"]
            }
        } else{
            for(tx_row in 1:nrow(taxadf)){
                tmp[tmp_count,"Group.1"]=cont
                tmp[tmp_count,taxadf[tx_row,"taxa.id"]] = taxadf[tx_row,cont]
                tmp[tmp_count,"bmark"]="expected"
                tmp[tmp_count,"bmark2"]=tmp[i,"bmark2"]
                tmp[tmp_count,"Group.2"]="cnt"
            } 
        }
        
    }
    tmp[is.na(tmp)] = 0 
    return(tmp)
}

In [28]:
calc_dist <- function(RA_in,CONTdf_in,hmo_in="std"){
    if(hmo_in=="hmo"){
        for(cnt in 1:nrow(RA_in)){
            if(is.na(RA_in[cnt,"Group.2"])){
                new_name=paste(RA_in[cnt,"Group.1"],RA_in[cnt,"bmark"],RA_in[cnt,"bmark2"],sep="-")
                RA_in[cnt,"new_name"] = new_name
            }else{
                new_name = paste(RA_in[cnt,"Group.1"],RA_in[cnt,"bmark"],RA_in[cnt,"Group.2"],sep="-")
                RA_in[cnt,"new_name"] = new_name
            }
        }
        rownames(RA_in)=RA_in$new_name
        RA_in = subset(RA_in, select = -c(new_name))
    } else{
        rownames(RA_in)=paste(RA_in$Group.1,RA_in$bmark,RA_in$bmark2,sep="-")
    }
    RA_in = subset(RA_in, select = -c(bmark2,Group.1,bmark,Group.2))

    taxa_list<-unique(paste(CONTdf_in$Family,CONTdf_in$Genus,sep=""))
    df_clean = RA_in[,colnames(RA_in) %in% taxa_list] #remove cols not expected

    df_out = vegdist(df_clean,method="bray")   
    return(df_out)
}

In [29]:
plot_dist_cluster <- function(df_in,ref_in){
    fname = paste(out_dir,"/img/hclust_summary_bc_allgroups_",ref_in,".png",sep="")
    png(fname,width = 900, height = 600)
    p = plot(hclust(df_in), cex = .9, main = "All Groups - Bray-Curtis Dissimilarity",
             xlab="Bioinformatic Variables", sub="")
    dev.off()
}

In [30]:
plot_dist_violin <- function(df_in,ref_in){
    df_in = as.data.frame(as.matrix(df_in))
    df_plot=data.frame()
    
    for (row in rownames(df_in)){
        splits = strsplit(row,"-")
        if(splits[[1]][2]=="expected"){
            next
        }else{
            df_plot[nrow(df_plot)+1,"dist"]=df_in[row,paste(splits[[1]][1],"expected","std",sep="-")]
            df_plot[nrow(df_plot),"name"]=paste(splits[[1]][2],splits[[1]][3],sep="-")
        }
    }
    
    p = ggplot(df_plot, aes(x=name, y=dist, color=name)) + 
            geom_violin(trim=FALSE) + 
            labs(color = df_plot$name) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            xlab("Benchmark ID") +
            ylab("Bray-Curtis Dissimilarity") +
            stat_summary(fun.data=mean_sdl, color="red")
    
    
    fig = ggarrange(p, 
              #labels = c("A"),
              #top = as.character(title_in),
              #common.legend = TRUE#,
              ncol = 1, nrow = 1)
    annotate_figure(fig, top = text_grob("Bray-Curtis Dissimilarity", color = "blue", face = "bold", size = 14)
    )
    ggsave(paste(out_dir,"/img/violin_bc_summary_allgroups_",ref_in,".jpg",sep=""))
    return(df_plot)

}

In [31]:
plot_dist_violin_hmo <- function(df_in,ref_in){
    df_in = as.data.frame(as.matrix(df_in))
    df_plot=data.frame()
    
    for (row in rownames(df_in)){
        splits = strsplit(row,"-")
        if(splits[[1]][2]=="expected"){
            next
        }else{
            df_plot[nrow(df_plot)+1,"dist"]=df_in[row,paste(splits[[1]][1],"expected","std",sep="-")]
            df_plot[nrow(df_plot),"name"]=splits[[1]][1]
            df_plot[nrow(df_plot),"Ext.Kit"]=splits[[1]][2]
            df_plot[nrow(df_plot),"Homo.Status"]=splits[[1]][3]
        }
    }
    
    p = ggplot(df_plot, aes(x=name, y=dist, color=Homo.Status)) + 
            geom_violin(trim=FALSE) + 
            facet_grid(Homo.Status ~ Ext.Kit) +
            labs(color = df_plot$name) +
            rotate_x_facet_text(angle = 90, align = 0.5) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            xlab("Benchmark ID") +
            ylab("Bray-Curtis Dissimilarity") +
            stat_summary(fun.data=mean_sdl, color="red")
    
    
    fig = ggarrange(p, 
              ncol = 1, nrow = 1)
    annotate_figure(fig, top = text_grob("Bray-Curtis Dissimilarity", color = "blue", face = "bold", size = 14)
    )
    ggsave(paste(out_dir,"/img/violin_bc_summary_allgroups_",ref_in,".jpg",sep=""))
    return(df_plot)

}

In [32]:
plot_corrmatrix <- function(df_in,contdf_in,sort_in){
    tmp = calc_dist(df_in[order(df_in[,sort_in]),],contdf_in)
    tmp = as.matrix(tmp)
    remove_list = list()
    for (rows in rownames(tmp)){
        if(strsplit(rows,"-")[[1]][2]=="expected"){
            remove_list=append(remove_list,rows)
        }
    }
    tmp_clean = tmp[!rownames(tmp) %in% remove_list,!colnames(tmp) %in% remove_list ]
    
    #cal sig
    p.mat <- cor.mtest(tmp_clean)$p
    
    #image with stats
    png(file = paste(out_dir,"/img/distmatrix_ext_sig_",sort_in,".jpg",sep=""))
    corrplot(as.matrix(tmp_clean), type = "upper",col=brewer.pal(n=8, name="RdYlBu"),
        title = "Correlation of  Dissimilarity (R): Ext Controls",
        tl.col = "black",
        mar = c(0,0,1,0), number.cex = 0.5, number.digits = 2,
        p.mat = p.mat, sig.level = 0.05, insig="blank")
    dev.off()
    
    #image without stats
    png(file = paste(out_dir,"/img/distmatrix_ext_",sort_in,".jpg",sep=""))
    
    corrplot(as.matrix(tmp_clean),type="upper",col=brewer.pal(n=8, name="RdYlBu"),
        title = "Correlation of  Dissimilarity (R): Ext Controls",
        tl.col = "black",
        mar = c(0,0,1,0), number.cex = 0.5, number.digits = 2)
    dev.off()
}
#Ref https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html

<h2> Statistics <h2>

In [33]:
tstat_paired_f1 <- function(f1_in,subgroup_in){
    
    df_out = data.frame()
    
    cal_list = c("F1")
    
    for (cal in cal_list){
    
        comb_list = list()
        for(kit in unique(f1_in[,subgroup_in])){
            if(sum(f1_in[,subgroup_in] == kit)>1)
                comb_list=append(comb_list,kit)
        }
        
        if(length(comb_list)>1){
            comb_list = combn(comb_list,2)
            count=1
        
            for (i in 1:(length(comb_list)/2)){

                tstatdf=data.frame()
                var1 = as.character(comb_list[count])
                var2 = as.character(comb_list[count+1])
                count=count+2

                df = f1_in %>%
                    filter(f1_in[,subgroup_in] == var1 | f1_in[,subgroup_in] == var2) %>%
                    select(subgroup_in, cal)

                tmp = t.test(get(cal) ~ get(subgroup_in), data = df)
                tstatdf[nrow(tstatdf)+1,"level"] = cal
                tstatdf[nrow(tstatdf),"comparison"] = paste(var1,var2,sep="_")
                tstatdf[nrow(tstatdf),"pval"] = tmp$p.value
                if(tmp$p.value<0.05){
                    tstatdf[nrow(tstatdf),"sig"] = "sig"
                } else{
                    tstatdf[nrow(tstatdf),"sig"] = "ns"
                }
                df_out = bind_rows(df_out,tstatdf)
            }        
            
        } else{
            next;
        }
    }
        
    return(df_out)
}

In [34]:
adonis_bc <- function(df_in){

    df_out=data.frame()
    
    st = adonis(df_in[,1] ~ name,
                data=df_in, permutations = 99, method="bray")
    df_out[nrow(df_out)+1,"comparison"] = "bray-curtis"
    df_out[nrow(df_out),"pval"] = as.numeric(st$aov.tab$"Pr(>F)"[1])
        
    if(df_out[nrow(df_out),"pval"]<0.05){
        df_out[nrow(df_out),"sig"] = "sig"
    } else{
        df_out[nrow(df_out),"sig"] = "ns"
    }   
    
    dist_s = vegdist(subset(dissdf_s,select = -c(bmark,Group.1,bmark2)),na.rm=TRUE)

        
    df_out[nrow(df_out)+1,"homo_s"] = anova(betadisper(dist_s,dissdf_s$bmark))$"Pr(>F)"[1]
    
    return(df_out)
}

<h3> Rel Abun Plots <h3>

In [35]:
addSmallLegend <- function(myPlot, pointSize = 1, textSize = 6, spaceLegend = 0.1) {
    myPlot +
        guides(shape = guide_legend(override.aes = list(size = pointSize)),
               color = guide_legend(override.aes = list(size = pointSize))) +
        theme(legend.title = element_text(size = textSize), 
              legend.text  = element_text(size = textSize),
              legend.key.size = unit(spaceLegend, "lines"))
}

In [36]:
phylum_colors <- c(
  "#CBD588", "#5F7FC7", "orange","#DA5724", "#508578", "#CD9BCD",
   "#AD6F3B", "#673770","#D14285", "#652926", "#C84248", 
  "#8569D5", "#5E738F","#D1A33D", "#8A7C64", "#599861"
)

rel_abun_plot <- function(df_in,x_in,y_in,x_lab_in,level_in,title_in,file_name){
    tmp = as.data.frame(unique(df_in[,c("Sample","Ext.Kit")]))
    
    p = ggplot(df_in, aes(x = get(x_in), y =get(y_in), fill = get(level_in))) + 
        theme(legend.position = "bottom", legend.key.width = unit(0.2,"cm"),
            axis.text.x = element_text(angle = 90, hjust = 1), legend.text = element_text(size = 8)) +
            facet_grid(Homo.Status~ .) +
            scale_fill_manual(values = phylum_colors) +
            geom_bar(stat = "identity") +
            xlab(x_in) +
            ylab("Relative Abundance \n") +
            ggtitle(title_in) +
            labs(fill = level_in)
    addSmallLegend(p)
    ggsave(paste(out_dir,"/img/",file_name,".jpg",sep=""))
}

In [37]:
adonis_rel <- function(df_in){
    df_out=data.frame()
    
    ext_list = unique(df_in$bmark)[-length(unique(df_in$bmark))]
    
    #analysis of each kit, to determine the sig of a homo on the control distance
    for (kit in ext_list){
        df_kit = subset(df_in,bmark==kit) #df only with kit
        cont_list = unique(df_kit$Group.1)
        
        for (cont in cont_list){
            df_cont = subset(df_kit,Group.1==cont) #df only with kit, and control
            df_cont = subset(df_cont,select=c("val","Group.2"))
            homo_list = unique(df_cont$Group.2)
            
            if(length(homo_list)>1){ #if there is more than one control, comparison can be performed
                
                st = adonis(val ~ Group.2,
                    data=df_cont, permutations = 99, method="bray")
                df_out[nrow(df_out)+1,"stat"] = "bray-curtis"
                df_out[nrow(df_out),"measure"] = paste(kit,cont,sep="-")
                df_out[nrow(df_out),"pval"] = as.numeric(st$aov.tab$"Pr(>F)"[1])
                
                if(df_out[nrow(df_out),"pval"]>0.05 | is.na(df_out[nrow(df_out),"pval"])){
                    df_out[nrow(df_out),"sig"] = "ns"
                } else{
                    df_out[nrow(df_out),"sig"] = "sig"
                }
            }
        }
    }
    
    #dist_s = vegdist(subset(dissdf_s,select = -c(bmark,Group.1,bmark2)),na.rm=TRUE)   
    #df_out[nrow(df_out)+1,"homo_s"] = anova(betadisper(dist_s,dissdf_s$bmark))$"Pr(>F)"[1]
    
    return(df_out)
}

<h3> Homog Stats <h3>

In [38]:
adonis_bc_metadf <- function(df_in){
    
    #create metadata df related to distance names
    metadf = as.data.frame(as.matrix(df_in))
    
    #Push out results
    for (rows in rownames(metadf)){
        splitname = strsplit(rows,"-")
        metadf[rows,"Subject.ID"] = splitname[[1]][1]
        metadf[rows,"Ext.Kit"] = splitname[[1]][2]
        metadf[rows,"Homo.Status"] = splitname[[1]][3]
        metadf[rows,"Ext.Robotics"] = as.character(metadf_clean[metadf[rows,"Ext.Kit"],"Ext.Robotics"])
        metadf[rows,"IR"] = as.character(metadf_clean[metadf[rows,"Ext.Kit"],"IR"])
        metadf[rows,"Mag.Col"] = as.character(metadf_clean[metadf[rows,"Ext.Kit"],"Mag.Col"])
    }
    metadf = subset(metadf,select=c(Subject.ID,Ext.Kit,Homo.Status,Ext.Robotics,IR,Mag.Col))
    return(metadf)
}

In [39]:
adonis_bc_stat <- function(cont_in,df_in,df_out,stat_in,strat_in="",stat_label){
    
    #Create metadata table
    metadf = adonis_bc_metadf(df_in)
    
    #Run adonis by control
    if(strat_in!=""){
        st = adonis(df_in ~ Ext.Kit, data=metadf, permutations = 99, strata=metadf[,strat_in])
    } else{
        st = adonis(df_in ~ get(stat_in), data=metadf, permutations = 99)
    }
    #save data to df
    df_out[nrow(df_out)+1,"stat"] = stat_label
    df_out[nrow(df_out),"measure"] = cont_in
    df_out[nrow(df_out),"pval"] = as.numeric(st$aov.tab$"Pr(>F)"[1])

    if(df_out[nrow(df_out),"pval"]>0.05 | is.na(df_out[nrow(df_out),"pval"])){
        df_out[nrow(df_out),"sig"] = "ns"
    } else{
        df_out[nrow(df_out),"sig"] = "sig"
    }
    return(df_out)
}

In [40]:
adonis_bc_hmo <- function(df_in){
    df_out=data.frame()

    for (cont in unique(df_in$Group.1)){
        df_sub = df_in

        df_sub$Group.1 <- as.character(df_sub$Group.1) #cont name
        df_sub$Group.2 <- as.character(df_sub$Group.2) #hmo method
        df_sub[is.na(df_sub)] = "std" #will change NA in Group.2 to std 

        df_clean = subset(df_sub,Group.1==cont & bmark!="expected") #remove expect control values

        #Cal dissimilarity between kits stratified by homo method
        df_cont = subset(df_clean)
        tmp = calc_dist(df_cont,CONTdf_s,"hmo")
        df_out = adonis_bc_stat(cont,tmp,df_out,"Ext.Kit","Homo.Status","all kits,Std/varied hmo")

        #Cal dissimilarity between kits, standard method only
        df_cont = subset(df_clean,Group.2=="std")
        tmp = calc_dist(df_cont,CONTdf_s,"hmo")
        df_out = adonis_bc_stat(cont,tmp,df_out,"Ext.Kit",stat_label="all kits,Std hmo")
        
        #Cal dissimilarity between kits, standard method only, automated platform
        df_cont = subset(df_clean,Group.2=="std" | Group.2=="Altered")
        tmp = calc_dist(df_cont,CONTdf_s,"hmo")
        df_out = adonis_bc_stat(cont,tmp,df_out,"Ext.Robotics", "Ext.Kit", stat_label="all kits,auto")

        #Cal dissimilarity between altered methods
        for (kit in unique(RAdf_oe$bmark)[-length(unique(RAdf_oe$bmark))]){ #Dont want to include "expected" 
            df_kit = subset(df_clean,bmark==kit)

            if(nrow(df_kit)>1){
                
                tmp = calc_dist(df_kit,CONTdf_s,"hmo")
                df_out = adonis_bc_stat(cont,tmp,df_out,"Homo.Status",stat_label=paste(kit,"-hmo",sep=""))
            }
        }
        
        #Cal dissimilarity between IR methods
        df_cont = subset(df_clean)
        tmp = calc_dist(df_cont,CONTdf_s,"hmo")
        df_out = adonis_bc_stat(cont,tmp,df_out,"Ext.Kit","IR","all kits,Std/varied hmo IR")
        
        #Cal dissimilarity between Mag.Col methods
        df_cont = subset(df_clean)
        tmp = calc_dist(df_cont,CONTdf_s,"hmo")
        df_out = adonis_bc_stat(cont,tmp,df_out,"Ext.Kit","Mag.Col","all kits,Std/varied hmo MC")
        
        df_sub = subset(RAdf_oe,bmark==kit)
    }

    return(df_out)
}

In [41]:
cal_ttest_hmo <-function(df_in){
    df_out = data.frame()
    
    #for each of the methods
    for(kit in unique(df_in$bmark)){ #each 
        tmp = subset(df_in,bmark==kit)

        #for each of the controls
        for(cont in unique(tmp$Subject.ID)){
            tmp2 = subset(tmp,Subject.ID==cont)

            #if there is more than one hmo instance, perform t-test
            if(sum(tmp2$bmark2=="hmo")>1){
                df_out[nrow(df_out)+1,"bmark"] = kit
                df_out[nrow(df_out),"level"] = "F1"
                df_out[nrow(df_out),"cont"] = cont
                df_out[nrow(df_out),"pval"] = t.test(F1~bmark2,tmp)$p.value
            }
        }
    }
    return(df_out)
}

<h3> Alpha and Beta <h3>

In [42]:
#Function for plotting alpha diversity
alpha_meas <- function(phy_in,alpha_list,sub_in,col_in,title,file_name,facet_input=""){
    
    
    if(facet_input!=""){
        p = plot_richness(phy_in, sub_in, col_in, measures=alpha_list)
        p_final = p + facet_wrap(~get(facet_input))
    } else{ 
        p_final <- plot_richness(phy_in, sub_in, col_in, measures=alpha_list)
    }
    
    fig = p_final + 
        geom_boxplot(data=p_final$data, aes(x=.data[[sub_in]], y=value, color=NULL), alpha=0.1) + 
        theme(legend.position="bottom")
    
    annotate_figure(fig, 
                   top = text_grob(title, color = "blue", face = "bold", size = 14))
    
    ggsave(paste(out_dir,"/img/alpha_plot_",file_name,".jpg",sep=""))
    
}
#https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html

In [43]:
plot_beta<-function(phyobj,type,dist_meas,colby,title_in,file_name,shape_in="",facet_in=""){
 
    ordu = ordinate(phyobj, type, dist_meas, weighted=TRUE)
    allGroupsColors<- c("grey0")
    allGroupsColors<-append(allGroupsColors,brewer.pal(n = 8, name = "Dark2"))
    allGroupsColors<-append(allGroupsColors,brewer.pal(n = 8, name = "Set2"))
    
    if(shape_in==""){
        p<-plot_ordination(phyobj, ordu, color=colby,title=title_in)+ 
            theme(legend.position = "bottom") + 
            geom_point(size = 3) +
            scale_color_manual(values = allGroupsColors) +
            scale_shape_manual(values = c(1,2,3,4,5,6,7,8,9,10,11,12,13,15, 16, 17))
    } else{
        p<-plot_ordination(phyobj, ordu, color=colby,title=title_in,shape=shape_in)+ 
            theme(legend.position = "bottom") +
            geom_point(size = 3) +
            scale_color_manual(values = allGroupsColors) +
            scale_shape_manual(values = c(1,2,3,4,5,6,7,8,9,10,11,12,13,15, 16, 17))
    } 
    
    if(facet_in==""){
        p_final = p 
    } else{
        p_final = p + facet_wrap(~get(facet_in))
    }
    
    addSmallLegend(p_final)

    ggsave(paste(out_dir,"/img/",file_name,".tiff",sep=""))
}

<h2> Run Production Code <h2>

<h3> Create objs <h3>

In [44]:
#Create the phyloseq object
tax_file = paste(out_dir,'taxonomic_classification/classify-sklearn_silva-132-99-nb-classifier.qza',sep="")
table_file = paste(out_dir,'denoising/feature_tables/merged_filtered.qza',sep="")
phy_obj = phylo_create(tax_file,table_file)  

#Rarefy samples
phy_r = rarefy_even_depth(phy_obj, rngseed=1, sample.size=20000, replace=F) 

#Create phy tree
random_tree = rtree(ntaxa(phy_r), rooted=TRUE, tip.label=taxa_names(phy_r))
phy_r_t = merge_phyloseq(phy_r, random_tree)

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1818 taxa and 351 samples ]
sample_data() Sample Data:       [ 351 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 1818 taxa by 7 taxonomic ranks ]


`set.seed(1)` was used to initialize repeatable random subsampling.

Please record this for your records so others can reproduce.

Try `set.seed(1); .Random.seed` for the full vector

...

69 samples removedbecause they contained fewer reads than `sample.size`.

Up to first five removed samples are: 


NTC-PC04925-H-12SC249366-PC04925-D-01SC249382-PC04925-F-01SC249406SC249416-PC04925-A-03

...

612OTUs were removed because they are no longer 
present in any sample after random subsampling


...



<h4> Data Stats <h4>

In [45]:
#Summarize data
temp = as.data.frame(sample_data(phy_obj))
table(temp$Sample.Type)
table(temp$Subject.ID)
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID), nrow),paste(out_dir,"/bench_files/counts_pre.csv",sep=""))

#Summarize data
temp = as.data.frame(sample_data(phy_r))
table(temp$Sample.Type)
table(temp$Subject.ID)
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID, temp$Homo.Status), nrow),paste(out_dir,"/bench_files/counts_post.csv",sep=""))
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID), nrow),paste(out_dir,"/bench_files/counts_post_hmo.csv",sep=""))



Ext_Control Seq_Control       Study 
        155          36         160 


      A00       A01       A02       A03       M16       M22       M98 NTC_Blank 
        5         5         5         5         5        24        49         2 
PCR_Blank     Water       Z00       Z05       Z06       Z10       Z11   human_1 
        2        50        55         5         5        21         2       111 


Ext_Control Seq_Control       Study 
        117          30         135 


    A00     A01     A02     A03     M16     M22     M98   Water     Z00     Z05 
      4       5       5       5       4      23      47      21      51       5 
    Z06     Z10     Z11 human_1 
      5      18       1      88 

<h3> Extraction Controls <h3>

<h4> Prepare Data <h4>

In [46]:
#Create glomed, subsetted obj
phy_ext = phy_subset_ext(phy_obj)

#Create OTUdf
OTUdf = otu_extract(phy_ext)

#Create TAXdf
TAXdf = tax_extract(phy_ext)

#Clean df's
OTUdf_clean = otu_tax_clean(OTUdf,TAXdf)

#Find averages reads
mean(rowSums(OTUdf))

[1] 54187.32

In [47]:
write.csv(sample_data(phy_ext),"save.csv")

In [48]:
#Change the out_dir for files to be saved
if(run_param=="dceg"){
    out_dir=paste(proj_dir,'output_dceg/',sep="")
}

In [49]:
#List of all variables to test
test_list = unique(sample_data(phy_ext)$Ext.Kit)
#test_list = test_list[7]
#test_list = test_list[1]

<h3> Standard Processing <h3>

In [50]:
#Create a merged dfs for all variables
mergedf_f1stats = data.frame()
merged_RAstats = data.frame()
dissdf_s = data.frame()
RAdf_merged = data.frame()

In [51]:
for (variable in test_list){
    #Subset dataframe
    OTUdf_clean2 = subset(OTUdf_clean,Ext.Kit==variable & Homo.Status=="Standard" & Subject.ID!="Ext_Blank")
    
    #Create TP,FP,FN df
    CONTdf_use = CONTdf_s
    F1df = bench_compare(OTUdf_clean2, CONTdf_use)
    
    #Create TP,FP,FN counts    
    bench_key = "std"
    F1df_counts = bench_counts(OTUdf_clean2,F1df,variable, bench_key)#swapped variable and benchkey from original
    temp = F1df_counts
    temp$sample_name = rownames(temp)
    rownames(temp) = c()
    mergedf_f1stats = rbind(mergedf_f1stats,temp)
    
    #Create violin plots for each control - counts
    bench_key = variable
    x_list = c("FN","TP","FP")
    y_labs = c("False negative","True positive", "False positive")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"counts_extstd")
    
    #Create violin plots for each control - summary
    bench_key = variable
    x_list = c("TAR","TDR","F1")
    y_labs = c("Taxoonmic Accuracy \n Rate","Taxonomic Detection \n Rate", "F-Measure")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"summary_extstd",-1,3)
    
    #calculate relative abundance
    RAdf = cal_relabun(OTUdf_clean2)
    RAdf$Sample_name = rownames(RAdf)
    RAdf_merged = rbind(RAdf_merged,RAdf)
    
    #Create scatter plots
    #Determine expected to observed scatter plots, normality, spearman coeff 
    title = paste("Scatterplots of controls: ", variable,sep="")
    
    print(nrow(RAdf))
    RAdf_stats = cal_EO_kit(RAdf,CONTdf_use,variable,"std",title)
    merged_RAstats = rbind(merged_RAstats,RAdf_stats)
    
    #average controls
    bench_key = "std"
    dissdf = create_av_RAdf(RAdf,variable,bench_key)
    dissdf_s = rbind(dissdf_s,dissdf)
}

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 2 rows containing non-finite values (stat_summary)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image



[1] 15
[1] "Z MagBead"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


[1] "Z MagBead"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


...

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] 9
[1] "Q DSP Virus"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


[1] "Q DSP Virus"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

Saving 6.67 x 6.67 in image



In [52]:
write.csv(merged_RAstats,paste(out_dir,"bench_files/RA_stats_kit_extstd.csv",sep=""))
write.csv(mergedf_f1stats,paste(out_dir,"bench_files/F1_vals_all_extstd.csv",sep=""))
write.csv(aggregate(mergedf_f1stats[, 3:8], list(mergedf_f1stats$Subject.ID,mergedf_f1stats$bmark), mean),paste(out_dir,"bench_files/f1_vals_mean_extstd.csv",sep=""))

<h4> Benchmarked Assessment<h4>

In [53]:
#Create dist df of all methods
Distdf_merged=data.frame()
#Silva
if(length(test_list)>1){
    RAdf_av = RAdf_addcontrols("CONTdf_s","dissdf_s")
    RAdf_clean = calc_dist(RAdf_av,CONTdf_s)
    plot_dist_cluster(RAdf_clean,"extstd")
    Distdf = plot_dist_violin(RAdf_clean,"extstd")
} 
Distdf_merged = bind_rows(Distdf_merged,Distdf)

Saving 6.67 x 6.67 in image



In [54]:
#is dist by method sig between all controls
adonis_stats = adonis_bc(Distdf_merged)
adonis_stats

,comparison,pval,sig,homo_s
,<chr>,<dbl>,<chr>,<dbl>
1,bray-curtis,0.82,ns,NA
2,NA,NA,NA,0.7571894


In [55]:
#Overall violin plots
x_list = c("TAR","TDR","F1")
y_labs = c("Taxonomic Accuracy \n Rate","Taxonomic Detection \nRate", "F-Measure")

#Create violin plots for each control across all methods
for(cont in unique(mergedf_f1stats$Subject.ID)){
    subdf = subset(mergedf_f1stats, Subject.ID==cont)
    
    for (rows in rownames(subdf)){ #add IR and magnetic/column data
        subdf[rows,"Mag.Col"] = metadata[subdf[rows,"sample_name"],"Mag.Col"]
        subdf[rows,"IR"] = metadata[subdf[rows,"sample_name"],"IR"]
    }
    
    plot_f1_violin(subdf,x_list,y_labs,"bmark",paste("Std Processing - ",cont,sep=""),"summary",-1,3)
    plot_f1_violin(subdf,x_list,y_labs,"IR",paste("Std Processing - ",cont,sep=""),"summary_ir",-1,3)
    plot_f1_violin(subdf,x_list,y_labs,"Mag.Col",paste("Std Processing - ",cont,sep=""),"summary_mc",-1,3)

}

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 2 rows containing non-finite values (stat_summary)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 2 rows containing non-finite values (stat_summary)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 2 rows containing non-finite values (stat_summary)."
Saving 6.67 x 6.67 in image

Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message:
"Removed 

In [56]:
#T-tests
#Run by control - not all kits have the same control
tstatdf_merged=data.frame()

for (cont in unique(mergedf_f1stats$Subject.ID)){
    subdf = subset(mergedf_f1stats,Subject.ID==cont)
    tstatdf = tstat_paired_f1(subdf,"bmark")
    if(nrow(tstatdf>1)){for(i in 1:nrow(tstatdf)){tstatdf[i,"cont"]=cont}}
    tstatdf_merged=bind_rows(tstatdf_merged,tstatdf)  
}
write.csv(tstatdf_merged,paste(out_dir,"bench_files/ttest_extstd.csv",sep=""))

Note: Using an external vector in selections is ambiguous.
i Use `all_of(subgroup_in)` instead of `subgroup_in` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.

Note: Using an external vector in selections is ambiguous.
i Use `all_of(cal)` instead of `cal` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [57]:
for (cont in unique(RAdf_merged$Subject.ID)){
    taxdf = create_taxadf(cont,CONTdf_s)
    cal_EO_cont(RAdf_merged,CONTdf_s,taxdf,cont,paste("Std Processing - ",cont,sep=""))
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

Saving 6.67 x 6.67 in image

`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 10 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 10 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 10 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 10 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 10 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 10 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 10 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 10 rows containi

In [58]:
#Create correlation matrices for extraction controls
RAdf_oe_save = RAdf_av
plot_corrmatrix(RAdf_av,CONTdf_s,"Group.1") #sort by kit
plot_corrmatrix(RAdf_av,CONTdf_s,"bmark") #sort by control

png 
  2

png 
  2

<h3> Blank assessment <h3>

<h3> Prepare Data <h3>

In [59]:
#Create glomed, subsetted obj
phy_blank = phy_subset_blank(phy_r)
phy_blank

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 233 taxa and 21 samples ]
sample_data() Sample Data:       [ 21 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 233 taxa by 7 taxonomic ranks ]

In [60]:
#Determine counts for blanks included, and those that passed filtering
tmp = ddply(sample_data(phy_obj), .(sample_data(phy_obj)$Ext.Kit, sample_data(phy_obj)$Subject.ID), nrow)
names(tmp) <- c("Ext.Kit", "Subject.ID", "Seq")
tmp
tmp = ddply(sample_data(phy_blank), .(sample_data(phy_blank)$Ext.Kit, sample_data(phy_blank)$Subject.ID), nrow)
names(tmp) <- c("Ext.Kit", "Subject.ID", "Passed QC")
tmp

Ext.Kit,Subject.ID,Seq
<fct>,<fct>,<int>
,A00,5
,A01,5
,A02,5
,A03,5
,NTC_Blank,2
,PCR_Blank,2
,Z05,5
,Z06,5
,Z11,2


Ext.Kit,Subject.ID,Passed QC
<fct>,<fct>,<int>
Q PowerMicrobiome,Water,5
Q PowerSoil,Water,7
Q QIAamp,Water,1
T MagMax,Water,1
Z MagBead,Water,7


In [61]:
#Calc rel abund
tmp = transform_sample_counts(phy_blank, function(x) {x/sum(x)} )
tmp = psmelt(tmp)
head(tmp)

,OTU,Sample,Abundance,Source.PCR.Plate,Run.ID,Project.ID,CGR.Sample.ID,Sample.Type,Sample.Des,Subject.ID,...,Unassigned,frac_filtered,Mag.Col,IR,Domain,Phylum,Class,Order,Family,Genus
,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,...,<int>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
2512,8ae518dbb29595b3f79214be0b589066,SC553068,0.8093541,PC22192,Run4,NP0084-MB6,SC553068,Ext_Control,Ext_Blank,Water,...,0,9.227570,Magnetic,Solution,D_0__Bacteria,D_1__Firmicutes,D_2__Bacilli,D_3__Bacillales,D_4__Listeriaceae,D_5__Listeria
2507,8ae518dbb29595b3f79214be0b589066,SC553048,0.7947894,PC22190,Run4,NP0084-MB6,SC553048,Ext_Control,Ext_Blank,Water,...,0,7.888065,Magnetic,Solution,D_0__Bacteria,D_1__Firmicutes,D_2__Bacilli,D_3__Bacillales,D_4__Listeriaceae,D_5__Listeria
2505,8ae518dbb29595b3f79214be0b589066,SC552975,0.7427614,PC22190,Run4,NP0084-MB6,SC552975,Ext_Control,Ext_Blank,Water,...,0,9.846112,Magnetic,Solution,D_0__Bacteria,D_1__Firmicutes,D_2__Bacilli,D_3__Bacillales,D_4__Listeriaceae,D_5__Listeria
2514,8ae518dbb29595b3f79214be0b589066,SC552961,0.7062563,PC22192,Run4,NP0084-MB6,SC552961,Ext_Control,Ext_Blank,Water,...,0,10.033773,Magnetic,Solution,D_0__Bacteria,D_1__Firmicutes,D_2__Bacilli,D_3__Bacillales,D_4__Listeriaceae,D_5__Listeria
822,333bbf9224442fc10cd497377d2b1d01,SC552952,0.4305763,PC22190,Run4,NP0084-MB6,SC552952,Ext_Control,Ext_Blank,Water,...,0,15.504106,Magnetic,Solution,D_0__Bacteria,D_1__Bacteroidetes,D_2__Bacteroidia,D_3__Bacteroidales,D_4__Bacteroidaceae,D_5__Bacteroides
1811,675c847bccbc53942ebb7b8cbb4efc4d,SC552966,0.3654507,PC22192,Run4,NP0084-MB6,SC552966,Ext_Control,Ext_Blank,Water,...,0,18.981414,Magnetic,Solution,D_0__Bacteria,D_1__Bacteroidetes,D_2__Bacteroidia,D_3__Bacteroidales,D_4__Prevotellaceae,D_5__Prevotella 9


In [62]:
#Create rel abun plots - summary
title = "Phyla Composition of Extraction Blanks \n Organized by Homogenization Method"
file_name = "relabun_extstd_blanks_summary"
rel_abun_plot(tmp,"Ext.Kit","Abundance","Ext.Kit","Phylum",title,file_name)

#Create rel abun plots - individual
phy_tmp <- phy_blank %>%
    tax_glom(taxrank = "Phylum") %>%                     # agglomerate at phylum level
    transform_sample_counts(function(x) {x/sum(x)} ) %>% # Transform to rel. abundance
    psmelt() %>%                                         # Melt to long format
    group_by(Ext.Kit) %>%
    arrange(Ext.Kit)
file_name = "relabun_extstd_blanks_indiv"
phy_tmp = phy_tmp[order(phy_tmp$Ext.Kit),]
rel_abun_plot(phy_tmp,"Sample","Abundance","Sample","Phylum",title,file_name)

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



<h3> Homogenization, Automation Assessment <h3>

In [63]:
#List of all variables to test
test_list = unique(sample_data(phy_r)$Ext.Kit)
test_list = test_list[c(-4,-5,-6)] #two protocols are not included in assessment

In [64]:
#Summarize data
temp = as.data.frame(sample_data(phy_obj))
table(temp$Sample.Type)
table(temp$Subject.ID)
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID, temp$Homo.Status), nrow),paste(out_dir,"/bench_files/counts_pre_hmo.csv",sep=""))

#Summarize data
temp = as.data.frame(sample_data(phy_r))
table(temp$Sample.Type)
table(temp$Subject.ID)
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID, temp$Homo.Status), nrow),paste(out_dir,"/bench_files/counts_post_hmo.csv",sep=""))


Ext_Control Seq_Control       Study 
        155          36         160 


      A00       A01       A02       A03       M16       M22       M98 NTC_Blank 
        5         5         5         5         5        24        49         2 
PCR_Blank     Water       Z00       Z05       Z06       Z10       Z11   human_1 
        2        50        55         5         5        21         2       111 


Ext_Control Seq_Control       Study 
        117          30         135 


    A00     A01     A02     A03     M16     M22     M98   Water     Z00     Z05 
      4       5       5       5       4      23      47      21      51       5 
    Z06     Z10     Z11 human_1 
      5      18       1      88 

In [65]:
for (variable in test_list){
    
    #Subset dataframe
    OTUdf_clean2 = subset(OTUdf_clean,Ext.Kit==variable & Homo.Status!="Standard" & Subject.ID!="Ext_Blank")
    
    #Create TP,FP,FN df
    CONTdf_use = CONTdf_s
    F1df = bench_compare(OTUdf_clean2, CONTdf_use)
    
    #Create TP,FP,FN counts    
    bench_key = "hmo"
    F1df_counts = bench_counts(OTUdf_clean2,F1df,variable, bench_key)#swapped variable and benchkey from original
    temp = F1df_counts
    temp$sample_name = rownames(temp)
    rownames(temp) = c()
    mergedf_f1stats = rbind(mergedf_f1stats,temp)

    #Create violin plots for each control - counts
    bench_key = variable
    x_list = c("FN","TP","FP")
    y_labs = c("False negative","True positive", "False positive")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"counts_exthmo")
    
    #Create violin plots for each control - summary
    bench_key = variable
    x_list = c("TAR","TDR","F1")
    y_labs = c("Taxonomic Accuracy Rate","Taxonomic Detection Rate", "F-Measure")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"summary_exthmo")
    
    #calculate relative abundance
    RAdf = cal_relabun(OTUdf_clean2)
    RAdf$Sample_name = rownames(RAdf)
    RAdf_merged=rbind(RAdf_merged,RAdf)

    #Create scatter plots
    #Determine expected to observed scatter plots, normality, spearman coeff 
    title = paste("Scatterplots of controls homogenization variation: \n", variable,sep="")
    EOdf_stats = cal_EO_kit(RAdf,CONTdf_use,variable,"hmo",title)    
    
    #Create averaged OTU df
    bench_key = "hmo"
    dissdf = create_av_RAdf(RAdf,variable,bench_key,"hmo") #swap order of bench and variable, include hmo for 2nd option
    dissdf_s = bind_rows(dissdf_s,dissdf)
    
    #save files
    write.csv(F1df_counts,paste(out_dir,"bench_files/F1_vals_exthmo_",variable,".csv",sep=""))#swap bench for variable
}

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "Z MagBead"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


[1] "Z MagBead"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

Saving 6.67 x 6.67 in image

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image



...

Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

Saving 6.67 x 6.67 in image

Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Saving 6.67 x 6.67 in image

Warning message in max(data$density):
"no non-missing arguments to m

[1] "T MagMax"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


[1] "T MagMax"


Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

Saving 6.67 x 6.67 in image



In [66]:
#Create distances matrix and plots
Distdf_merged_hmo=data.frame()
if(length(test_list)>1){
    #add controls
    RAdf_oe = RAdf_addcontrols("CONTdf_s","dissdf_s","hmo")
    RAdf_clean = calc_dist(RAdf_oe,CONTdf_s,"hmo")
    plot_dist_cluster(RAdf_clean,"exthmo")
    Distdf = plot_dist_violin_hmo(RAdf_clean,"exthmo")
} 
Distdf_merged_hmo = bind_rows(Distdf_merged_hmo,Distdf)

Warning message in `[<-.factor`(`*tmp*`, thisvar, value = 0):
"invalid factor level, NA generated"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, dat

In [67]:
#Run stats on distances of relative abundances
ttestdf_hmo = cal_ttest_hmo(mergedf_f1stats)
write.csv(ttestdf_hmo,paste(out_dir,'/bench_files/ttest_hmo.csv',sep=""))

metadf_phy_r = as.data.frame(sample_data(phy_r))
metadf_clean = subset(metadf_phy_r,select=c("Ext.Kit","Ext.Robotics","IR","Mag.Col"))
metadf_clean= metadf_clean[!duplicated(metadf_clean$Ext.Kit), ]
rownames(metadf_clean)=metadf_clean$Ext.Kit
metadf_clean$Ext.Robotics <- as.character(metadf_clean$Ext.Robotics)
metadf_clean$IR <- as.character(metadf_clean$IR)
metadf_clean$Mag.Col <- as.character(metadf_clean$Mag.Col)


ad_stat = adonis_bc_hmo(RAdf_oe)
write.csv(ad_stat,paste(out_dir,"bench_files/adtest_summary.csv",sep=""))

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

Set of permutations < 'minperm'. Generating entire set.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutations < 'minperm'. Generating entire set.

'nperm' >= set of all permutations: complete enumeration.

Set of permutat

In [68]:
#Overall violin plots
x_list = c("TAR","TDR","F1")
y_labs = c("Taxonomic Accuracy Rate","Taxonomic Detection Rate", "F-Measure")

#add homo.status
mergedf_f1stats_hmo=mergedf_f1stats
for(row in 1:nrow(mergedf_f1stats)){
    sample = mergedf_f1stats[row,"sample_name"]
    mergedf_f1stats_hmo[row,"Homo.Status"] = metadata[sample,"Homo.Status"]
    mergedf_f1stats_hmo[row,"Mag.Col"] = metadata[sample,"Mag.Col"]
    mergedf_f1stats_hmo[row,"IR"] = metadata[sample,"IR"]
}
#plot TAR,TDR,F1 individually, faceted  by homo status
plot_f1_violin_hmo(mergedf_f1stats_hmo,x_list,y_labs,"Homo.Status","Ext Protocols Homogenization","summary",1)
plot_f1_violin_hmo(mergedf_f1stats_hmo,x_list,y_labs,"bmark","Ext Protocols Homogenization","summary_kit")
plot_f1_violin_hmo(mergedf_f1stats_hmo,x_list,y_labs,"bmark","Ext Protocols Homogenization","summary_inverted",3)
plot_f1_violin_hmo(mergedf_f1stats_hmo,x_list,y_labs,"Mag.Col","Ext Protocols Homogenization","summary_mc")
plot_f1_violin_hmo(mergedf_f1stats_hmo,x_list,y_labs,"IR","Ext Protocols Homogenization","summary_ir")

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 2 rows containing non-finite values (stat_summary)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message in max(data$density):
"no non-missing arguments to max

Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Warning message:
"Removed 2 rows containing missing values (geom_segment)."
Warning message:
"Removed 3 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:


Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has

Warning message:
"Removed 2 rows containing missing values (geom_segment)."
Warning message:
"Removed 2 rows containing missing values (geom_segment)."
Warning message:
"Removed 2 rows containing missing values (geom_segment)."
Warning message:
"Removed 2 rows containing missing values (geom_segment)."
Warning message:
"Removed 1 rows containing missing values (geom_segment)."
Saving 6.67 x 6.67 in image

Warning message:
"Removed 2 rows containing non-finite values (stat_ydensity)."
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:
replacement has 1 row, data has 0"
Warning message in max(data$density):
"no non-missing arguments to max; returning -Inf"
Warning message:
"Computation failed in `stat_ydensity()`:


<h3> Human Assessment <h3>

In [69]:
#Create object
sub_list<-c("human_1")
phy_hum<-subset_samples(phy_obj, Sample.Des %in% sub_list)
phy_hum

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1818 taxa and 111 samples ]
sample_data() Sample Data:       [ 111 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 1818 taxa by 7 taxonomic ranks ]

In [70]:
#Alpha div before rarefaction
alpha_list = c("Observed","ACE","Shannon","Simpson","InvSimpson")
title = "Alpha Diversity Plot of Unraryfied Data"
file_name = "hum_unrare_extkit"
alpha_meas(phy_hum,alpha_list,"Ext.Kit","Ext.Kit",title,file_name)

Warning message:
"Removed 7 rows containing non-finite values (stat_boxplot)."
Saving 6.67 x 6.67 in image



In [71]:
#subset samples
phy_hum <-subset_samples(phy_r_t, Sample.Des %in% sub_list)
phy_hum

#subset for standard hmo
phy_hum_std = subset_samples(phy_hum, Homo.Status=="Standard")
phy_hum_std

#susbet for hmo
phy_hum_hmo = subset_samples(phy_hum, Homo.Status!="Standard")
phy_hum_hmo

#subset for single samples
#sample_list = c("SC553000","SC553034","SC553038","SC553042","SC553046")#34 sucks
#sample_list = c("SC553000","SC553050","SC553038","SC553042","SC553046") #50 sucks
sample_list = c("SC553000","SC553084","SC553038","SC553042","SC553046") 
phy_hum_sub = subset_samples(phy_hum_hmo, CGR.Sample.ID %in% sample_list)
phy_hum_sub

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1206 taxa and 88 samples ]
sample_data() Sample Data:       [ 88 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 1206 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1206 tips and 1205 internal nodes ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1206 taxa and 69 samples ]
sample_data() Sample Data:       [ 69 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 1206 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1206 tips and 1205 internal nodes ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1206 taxa and 19 samples ]
sample_data() Sample Data:       [ 19 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 1206 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1206 tips and 1205 internal nodes ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1206 taxa and 5 samples ]
sample_data() Sample Data:       [ 5 samples by 27 sample variables ]
tax_table()   Taxonomy Table:    [ 1206 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1206 tips and 1205 internal nodes ]

In [72]:
#Summarize data
temp = as.data.frame(sample_data(phy_hum_std))
table(temp$Sample.Type)
table(temp$Subject.ID)
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID), nrow),paste(out_dir,"/bench_files/counts_hum_std.csv",sep=""))

#Summarize data
temp = as.data.frame(sample_data(phy_hum_hmo))
table(temp$Sample.Type)
table(temp$Subject.ID)
ddply(temp, .(temp$Ext.Kit, temp$Subject.ID), nrow)
write.csv(ddply(temp, .(temp$Ext.Kit, temp$Subject.ID), nrow),paste(out_dir,"/bench_files/counts_hum_hmo.csv",sep=""))


Study 
   69 


human_1 
     69 


Study 
   19 


human_1 
     19 

temp$Ext.Kit,temp$Subject.ID,V1
<fct>,<fct>,<int>
Q PowerMicrobiome,human_1,11
Q PowerSoil,human_1,4
Q PowerSoil Pro,human_1,3
Z MagBead,human_1,1


In [73]:
#Alpha diversity, other metrics
alpha_list = c("Observed","ACE","Shannon","Simpson","InvSimpson")
title = "Alpha Diversity Plot of Unraryfied Data"
file_name = "hum_rare_extkit"
alpha_meas(phy_hum,alpha_list,"Ext.Kit","Ext.Kit",title,file_name)

#std only
alpha_list = c("Observed","ACE","Shannon","Simpson","InvSimpson")
title = "Alpha Diversity Plot of Raryfied Data: \n Std Processing"
file_name = "hum_std_extkit"
alpha_meas(phy_hum_std,alpha_list,"Ext.Kit","Ext.Kit",title,file_name)

#hmo only
alpha_list = c("Observed","ACE","Shannon","Simpson","InvSimpson")
title = "Alpha Diversity Plot of Raryfied Data: \n Non-Standard Processing"
file_name = "hum_hmo_extkit"
alpha_meas(phy_hum_hmo,alpha_list,"Ext.Kit","Ext.Kit",title,file_name)

#break out each metric
alpha_list = c("Observed","ACE","Shannon","Simpson","InvSimpson")
for (alpha in alpha_list){
    title = paste("Alpha Diversity Plot of Raryfied Data: \n",alpha," - All Processing Methods",sep="")
    file_name = paste("hum_rare_extkit_",alpha,sep="")
    alpha_meas(phy_hum,alpha,"Ext.Kit","Ext.Kit",title,file_name,"Homo.Status")
}

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [ ]:
fig = plot_anova_diversity(phy_hum_std,method=c("richness","simpson","shannon"),grouping_column="Ext.Kit")
fname_in = "alpha_plot_stats_hum_extkit.jpg"
title_in = "Alpha Diversity Plot \n Human, Std processing"
annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                    top = text_grob(title_in, color = "black", face = "bold", size = 14))
ggsave(paste(out_dir,"/img/",fname_in,sep=""))

In [74]:
#Beta diveresity

#plot all samples
title = "Beta Diversity Plot of Raryfied Data: \n All samples"
file_name = "beta_all_subj&kit"
plot_beta(phy_r_t,"PCoA","bray","Subject.ID",title,file_name,"Ext.Kit")

#Plot extracted material only
phy_sub = subset_samples(phy_r_t,Sample.Type!="Seq_Control")

title = "Beta Diversity Plot of Raryfied Data: \n Extacted samples"
file_name = "beta_all_strat_st"
plot_beta(phy_sub,"PCoA","bray","Homo.Method",title,file_name,"Ext.Kit","Subject.ID")

title = "Beta Diversity Plot of Raryfied Data: \n Extacted samples"
file_name = "beta_all_strat_hm"
plot_beta(phy_sub,"PCoA","bray","Subject.ID",title,file_name,"Ext.Kit","Homo.Method")

title = "Beta Diversity Plot of Raryfied Data: \n Extacted samples"
file_name = "beta_all_strat_ek"
plot_beta(phy_sub,"PCoA","bray","Subject.ID",title,file_name,"Homo.Method","Ext.Kit")


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [123]:
beta_nmds <- function(title_in,phy_in,group_in,filename_in){
    
    ord.res = ordination_MS(phy_in,method="NMDS",grouping_column=group_in)
    
    fname = paste(out_dir,"/img/",filename_in,sep="")
    fig = plot.ordination(ord.res,method="NMDS",show.pvalues=T)
    annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                    top = text_grob(title_in, color = "black", face = "bold", size = 14))
    ggsave(fname)   
}

In [131]:
title = "Beta Diversity Plot of Raryfied Data: \nAll samples"
filename = "beta_nmds_all_subid.png"
beta_nmds(title,phy_r,"Subject.ID",filename)

sub_list<-c("Ext_Control","Study")
phy_sub<-subset_samples(phy_obj, Sample.Type %in% sub_list)
title = "Beta Diversity Plot of Raryfied Data: \nExtracted samples"
filename = "beta_nmds_all_extkit.png"
beta_nmds(title,phy_sub,"Ext.Kit",filename)

title = "Beta Diversity Plot of Raryfied Data: \nHuman, std processing"
filename = "beta_nmds_human_extkit.png"
beta_nmds(title,phy_hum_std,"Ext.Kit",filename)

Square root transformation
Wisconsin double standardization
Run 0 stress 0.1325511 
Run 1 stress 0.1380476 
Run 2 stress 0.1374752 
Run 3 stress 0.1492127 
Run 4 stress 0.1392861 
Run 5 stress 0.1442522 
Run 6 stress 0.1435201 
Run 7 stress 0.1493308 
Run 8 stress 0.1434865 
Run 9 stress 0.1491837 
Run 10 stress 0.1469185 
Run 11 stress 0.1358245 
Run 12 stress 0.1492312 
Run 13 stress 0.1438988 
Run 14 stress 0.1492161 
Run 15 stress 0.1424806 
Run 16 stress 0.1435601 
Run 17 stress 0.1484374 
Run 18 stress 0.1405059 
Run 19 stress 0.143655 
Run 20 stress 0.138999 
*** No convergence -- monoMDS stopping criteria:
    18: stress ratio > sratmax
     2: scale factor of the gradient < sfgrmin
                     groups      p_value label
Water-human_1 Water-human_1 3.001837e-18   ***
M98-Water         M98-Water 2.812441e-17   ***
A01-human_1     A01-human_1 9.101305e-14   ***
Water-Z00         Water-Z00 3.513261e-13   ***
Z05-human_1     Z05-human_1 1.822713e-12   ***
Z06-human_1     Z0

Saving 6.67 x 6.67 in image



Square root transformation
Wisconsin double standardization
Run 0 stress 0.0001807565 
Run 1 stress 0.0001053839 
... New best solution
... Procrustes: rmse 0.04010368  max resid 0.5305595 
Run 2 stress 0.0001451371 
... Procrustes: rmse 0.04186765  max resid 0.4421701 
Run 3 stress 9.587129e-05 
... New best solution
... Procrustes: rmse 0.02251705  max resid 0.3471259 
Run 4 stress 0.000145203 
... Procrustes: rmse 0.049877  max resid 0.6485998 
Run 5 stress 0.0001570857 
... Procrustes: rmse 0.04871442  max resid 0.5168882 
Run 6 stress 0.000110013 
... Procrustes: rmse 0.04328175  max resid 0.6636711 
Run 7 stress 0.0001389758 
... Procrustes: rmse 0.04979244  max resid 0.6520145 
Run 8 stress 0.0001652426 
... Procrustes: rmse 0.04863025  max resid 0.6174644 
Run 9 stress 0.0001426285 
... Procrustes: rmse 0.03986238  max resid 0.4060874 
Run 10 stress 0.0001301682 
... Procrustes: rmse 0.04905901  max resid 0.5477023 
Run 11 stress 0.0001689039 
... Procrustes: rmse 0.05017908  m

Warning message in metaMDS(veganifyOTU(physeq), distance, ...):
"stress is (nearly) zero: you may have insufficient data"


                                                             groups
Q DSP Virus-Q QIAamp                           Q DSP Virus-Q QIAamp
Q PowerSoil Pro-Q QIAamp                   Q PowerSoil Pro-Q QIAamp
Q DSP Virus-Q PowerMicrobiome         Q DSP Virus-Q PowerMicrobiome
Q QIAamp-Z MagBead                               Q QIAamp-Z MagBead
Q PowerMicrobiome-Q PowerSoil Pro Q PowerMicrobiome-Q PowerSoil Pro
Q PowerMicrobiome-Z MagBead             Q PowerMicrobiome-Z MagBead
Q PowerSoil-Q QIAamp                           Q PowerSoil-Q QIAamp
Q PowerMicrobiome-Q PowerSoil         Q PowerMicrobiome-Q PowerSoil
Q QIAamp-T MagMax                                 Q QIAamp-T MagMax
Q DSP Virus-Q PowerSoil                     Q DSP Virus-Q PowerSoil
Q PowerMicrobiome-T MagMax               Q PowerMicrobiome-T MagMax
                                       p_value label
Q DSP Virus-Q QIAamp              6.379709e-05   ***
Q PowerSoil Pro-Q QIAamp          1.128763e-04   ***
Q DSP Virus-Q PowerMicrob

Saving 6.67 x 6.67 in image



Square root transformation
Wisconsin double standardization
Run 0 stress 0.1275733 
Run 1 stress 0.1601267 
Run 2 stress 0.1802951 
Run 3 stress 0.1275733 
... Procrustes: rmse 1.76601e-05  max resid 6.943519e-05 
... Similar to previous best
Run 4 stress 0.1456628 
Run 5 stress 0.1425107 
Run 6 stress 0.1601457 
Run 7 stress 0.1275733 
... Procrustes: rmse 3.452728e-05  max resid 0.000158068 
... Similar to previous best
Run 8 stress 0.1847655 
Run 9 stress 0.1425892 
Run 10 stress 0.1554387 
Run 11 stress 0.1456634 
Run 12 stress 0.1275734 
... Procrustes: rmse 6.463918e-05  max resid 0.0002877033 
... Similar to previous best
Run 13 stress 0.142617 
Run 14 stress 0.1470019 
Run 15 stress 0.144289 
Run 16 stress 0.1554211 
Run 17 stress 0.1275889 
... Procrustes: rmse 0.0007813278  max resid 0.005083024 
... Similar to previous best
Run 18 stress 0.1282943 
Run 19 stress 0.1275735 
... Procrustes: rmse 3.856903e-05  max resid 0.0001844422 
... Similar to previous best
Run 20 stress 0

Saving 6.67 x 6.67 in image



<h2> Comparative Analysis <h2>

In [75]:
subset_df <- function(tmp_in,max_in=1){
    df_out=data.frame()
    max_count=max_in
    #for each kit add rows until max
    for (kit in unique(tmp_in$Ext.Kit)){
        for(cont in unique(tmp_in$Subject.ID)){
            tmp2 = subset(tmp_in,Ext.Kit==kit&Subject.ID==cont)
            for(row in 1:max_count){
                df_out = rbind(df_out,tmp2[row,])   
            } 
        }    
    }    
    return(df_out)
}

In [76]:
adonis_stat <- function(dist_in,test_in,test_name){    
    distdf_sub = dist_in[,1:(ncol(dist_in)-5)]
    distdf = vegdist(distdf_sub,method="bray")

    metadata$CGR.ID = rownames(metadata)
    metadata_sub = subset(metadata,CGR.ID %in% rownames(as.data.frame(as.matrix(distdf))))

    #perform comparisons
    #print(adonis(distdf ~ get(test_in), data=metadata_sub, permutations = 99)$aov.tab$"Pr(>F)"[1])
    df_out = data.frame()
    df_out[nrow(df_out)+1,"pval"]=adonis(distdf ~ get(test_in), data=metadata_sub, permutations = 99)$aov.tab$"Pr(>F)"[1]
    if(df_out[nrow(df_out),"pval"]>.05) {df_out[nrow(df_out),"sig"]="ns"} else{df_out[nrow(df_out),"sig"]="sig"}
    df_out[nrow(df_out),"testing"]=test_in
    df_out[nrow(df_out),"description"]=test_name
    return(df_out)
}

In [77]:
#Prepare samples
#Create OTUdf
OTUdf = otu_extract(phy_r)

#Create TAXdf
TAXdf = tax_extract(phy_r)

#Clean df's
OTUdf_clean = otu_tax_clean(OTUdf,TAXdf)
RAdf = cal_relabun(OTUdf_clean)
RAdf_sub = subset(RAdf,Subject.ID!="Water")

In [78]:
#stats
stats_df=data.frame()

#group 1
#all kits, D6300, Ext,Kit
Subject.List = c("Z00","M98")
tmp = subset(RAdf_sub, Subject.ID %in% Subject.List&Homo.Status=="Standard")
distdf = subset_df(tmp)
stats_df = (adonis_stat(distdf,"Ext.Kit","Z00,M98 by Ext.Kit for all methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"Subject.ID","Z00,M98 by Subject.ID for all methods"))


#group 2
#all kits, D6300, Ext,Kit
Ext.List = c("Q DSP Virus","Q PowerMicrobiome", "Q PowerSoil Pro","Q PowerSoil","Q QIAamp")
Subject.List = c("Z00","M22","M98")
tmp = subset(RAdf_sub, Ext.Kit %in% Ext.List & Subject.ID %in% Subject.List&Homo.Status=="Standard")
distdf = subset_df(tmp)
stats_df = rbind(stats_df,adonis_stat(distdf,"Ext.Kit","Z00,M22,M98 by Ext.Kit for all qiagen methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"Subject.ID","Z00,M22,M98 by Subject.ID for all qiagen methods"))

#group 3
#select methods, Z00/human_1, Ext.Kit/Subject.ID,Homo.Status
Ext.List = c("Q PowerMicrobiome", "Q PowerSoil Pro","Q PowerSoil","Z MagBead")
Subject.List = c("Z00","human_1")
tmp = subset(RAdf_sub, Ext.Kit %in% Ext.List & Subject.ID %in% Subject.List&Homo.Status=="Standard")
distdf = subset_df(tmp)
tmp = subset(RAdf_sub, Ext.Kit %in% Ext.List & Subject.ID %in% Subject.List&Homo.Status!="Standard")
distdf2 = subset_df(tmp)
distdf=rbind(distdf,distdf2)
distdf$Homo.Status = as.character(distdf$Homo.Status)
for(row in 1:nrow(distdf)){
    if(distdf[row,"Homo.Status"]!="Standard"){
        distdf[row,"Homo.Status"]="Variation"
    }
}
stats_df = rbind(stats_df,adonis_stat(distdf,"Ext.Kit","Z00&human_1 by Ext.Kit for group3 methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"Subject.ID","Z00&human_1 by Subject.ID for group3 methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"Homo.Status","Z00&human_1 by Subject.ID for group3 methods"))


#group 4
Ext.List = c("Q PowerMicrobiome", "Q PowerSoil Pro","Q PowerSoil")
Subject.List = c("Z00","Z10","M22","M98")
tmp = subset(RAdf_sub, Ext.Kit %in% Ext.List & Subject.ID %in% Subject.List&Homo.Status=="Standard")
distdf = subset_df(tmp)
stats_df = rbind(stats_df,adonis_stat(distdf,"Ext.Kit","Z00,Z10,M22,M98 by Ext.Kit for group4 methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"Subject.ID","Z00,Z10,M22,M98 by Subject.ID for group4 methods"))

#group 5
Ext.List = c("Q PowerMicrobiome", "Q PowerSoil Pro","Q PowerSoil", "Z MagBead","Q QIAamp")
Subject.List = c("human_1")
tmp = subset(RAdf_sub, Ext.Kit %in% Ext.List & Subject.ID %in% Subject.List&Homo.Status=="Standard")
distdf = subset_df(tmp,5)
stats_df = rbind(stats_df,adonis_stat(distdf,"Ext.Kit","human by Ext.Kit for group5 methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"IR","human by IR for group5 methods"))
stats_df = rbind(stats_df,adonis_stat(distdf,"Mag.Col","human by Mag.Col for group5 methods"))

write.csv(stats_df,paste(out_dir,"bench_files/adtest_summary_groups.csv",sep=""))

In [152]:
network_plotting <- function(phy_in,color_in,title_in,fname_in,shape_in=NULL){ 
    #https://joey711.github.io/phyloseq/plot_network-examples.html
    ig = make_network(phy_in, dist.fun="bray", max.dist=0.3)
    fig = plot_network(ig, physeq=phy_in, color=color_in, line_weight=0.4, label=NULL,shape=shape_in) +
        scale_shape_manual(values = c(15,16,17,18,3,8,0,1,2))
    #plot_net(phy_hum_std, color="Ext.Kit", distance="bray") #alt, not used

    annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                       top = text_grob(title_in, color = "black", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/",fname_in,".jpg",sep=""))
}

In [153]:
#Create network tree
title="Network created using Bray-Curtis dissimilarity matrix: \n Human Samples, Std Processing"
network_plotting(phy_hum_std,"Run.ID",title,"net_human_std_runid.jpg")

title="Network created using Bray-Curtis dissimilarity matrix: \n Human Samples, Std Processing"
network_plotting(phy_hum_std,"Ext.Kit",title,"net_human_std_extkit.jpg","Run.ID")

title="Network created using Bray-Curtis dissimilarity matrix: \n All Samples, All Processing"
network_plotting(phy_r_t,"Subject.ID",title,"net_all_subjid.jpg","Ext.Kit")

title="Network created using Bray-Curtis dissimilarity matrix: \n All Samples, All Processing"
network_plotting(phy_r_t,"Ext.Kit",title,"net_all_extkit.jpg")

Warning message:
"attributes are not identical across measure variables; they will be dropped"
Saving 6.67 x 6.67 in image

Warning message:
"attributes are not identical across measure variables; they will be dropped"
Saving 6.67 x 6.67 in image

Warning message:
"attributes are not identical across measure variables; they will be dropped"
Saving 6.67 x 6.67 in image

Warning message:
"attributes are not identical across measure variables; they will be dropped"
Saving 6.67 x 6.67 in image

